<a href="https://colab.research.google.com/github/LinLat12/expense-analysis-python/blob/main/Myportfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
"データ分析ポートフォリオを始めます"
import pandas as pd

# 架空の経費データを作成
data = {
"現場ID": ["A01", "A01", "A02", "A02", "A03", "A03", "A04", "A04"],
"月": ["2025-01", "2025-02", "2025-01", "2025-02", "2025-01", "2025-02", "2025-01", "2025-02"],
"費目": ["材料費", "外注費", "材料費", "交通費", "材料費", "外注費", "交通費", "材料費"],
"現場申請額": [120000, 300000, 150000, 20000, 180000, 250000, 15000, 160000],
"システム金額": [118000, 300000, 150000, 18000, 175000, 260000, 15000, 155000],
"取引先": ["AAA商事", "BBB建設", "AAA商事", "交通会社", "CCC工業", "BBB建設", "交通会社", "AAA商事"]
}

df = pd.DataFrame(data)

# データを表示
df

,現場ID,月,費目,現場申請額,システム金額,取引先
0,A01,2025-01,材料費,120000,118000,AAA商事
1,A01,2025-02,外注費,300000,300000,BBB建設
2,A02,2025-01,材料費,150000,150000,AAA商事
3,A02,2025-02,交通費,20000,18000,交通会社
4,A03,2025-01,材料費,180000,175000,CCC工業
5,A03,2025-02,外注費,250000,260000,BBB建設
6,A04,2025-01,交通費,15000,15000,交通会社
7,A04,2025-02,材料費,160000,155000,AAA商事


In [11]:
# 差異金額を計算
df["差異金額"] = df["現場申請額"] - df["システム金額"]

# 差異率を計算
df["差異率"] = df["差異金額"] / df["システム金額"]

df

,現場ID,月,費目,現場申請額,システム金額,取引先,差異金額,差異率
0,A01,2025-01,材料費,120000,118000,AAA商事,2000,0.016949
1,A01,2025-02,外注費,300000,300000,BBB建設,0,0.000000
2,A02,2025-01,材料費,150000,150000,AAA商事,0,0.000000
3,A02,2025-02,交通費,20000,18000,交通会社,2000,0.111111
4,A03,2025-01,材料費,180000,175000,CCC工業,5000,0.028571
5,A03,2025-02,外注費,250000,260000,BBB建設,-10000,-0.038462
6,A04,2025-01,交通費,15000,15000,交通会社,0,0.000000
7,A04,2025-02,材料費,160000,155000,AAA商事,5000,0.032258


In [12]:
# 確認対象フラグを作成
# 条件：差異金額が ±5,000円以上 または 差異率が ±5%以上
df["要確認"] = (
(df["差異金額"].abs() >= 5000) |
(df["差異率"].abs() >= 0.05)
)

df

,現場ID,月,費目,現場申請額,システム金額,取引先,差異金額,差異率,要確認
0,A01,2025-01,材料費,120000,118000,AAA商事,2000,0.016949,False
1,A01,2025-02,外注費,300000,300000,BBB建設,0,0.000000,False
2,A02,2025-01,材料費,150000,150000,AAA商事,0,0.000000,False
3,A02,2025-02,交通費,20000,18000,交通会社,2000,0.111111,True
4,A03,2025-01,材料費,180000,175000,CCC工業,5000,0.028571,True
5,A03,2025-02,外注費,250000,260000,BBB建設,-10000,-0.038462,True
6,A04,2025-01,交通費,15000,15000,交通会社,0,0.000000,False
7,A04,2025-02,材料費,160000,155000,AAA商事,5000,0.032258,True


In [13]:
# 要確認データを現場IDごとに件数集計
df_summary = (
df[df["要確認"] == True]
.groupby("現場ID")
.size()
.reset_index(name="要確認件数")
)

df_summary

,現場ID,要確認件数
0,A02,1
1,A03,2
2,A04,1


In [14]:
# 要確認レベルを作成
df_summary["要確認レベル"] = df_summary["要確認件数"].apply(
lambda x: "2以上" if x >= 2 else ("1" if x == 1 else "0")
)

df_summary

,現場ID,要確認件数,要確認レベル
0,A02,1,1
1,A03,2,2以上
2,A04,1,1


In [15]:
# 元データ(df) と 現場別サマリ(df_summary)を結合
df_merged = df.merge(
df_summary,
on="現場ID",
how="left"
)

df_merged

,現場ID,月,費目,現場申請額,システム金額,取引先,差異金額,差異率,要確認,要確認件数,要確認レベル
0,A01,2025-01,材料費,120000,118000,AAA商事,2000,0.016949,False,NaN,NaN
1,A01,2025-02,外注費,300000,300000,BBB建設,0,0.000000,False,NaN,NaN
2,A02,2025-01,材料費,150000,150000,AAA商事,0,0.000000,False,1.0,1
3,A02,2025-02,交通費,20000,18000,交通会社,2000,0.111111,True,1.0,1
4,A03,2025-01,材料費,180000,175000,CCC工業,5000,0.028571,True,2.0,2以上
5,A03,2025-02,外注費,250000,260000,BBB建設,-10000,-0.038462,True,2.0,2以上
6,A04,2025-01,交通費,15000,15000,交通会社,0,0.000000,False,1.0,1
7,A04,2025-02,材料費,160000,155000,AAA商事,5000,0.032258,True,1.0,1


In [16]:
df_merged["要確認件数"] = df_merged["要確認件数"].fillna(0).astype(int)
df_merged["要確認レベル"] = df_merged["要確認レベル"].fillna("0")

df_merged

,現場ID,月,費目,現場申請額,システム金額,取引先,差異金額,差異率,要確認,要確認件数,要確認レベル
0,A01,2025-01,材料費,120000,118000,AAA商事,2000,0.016949,False,0,0
1,A01,2025-02,外注費,300000,300000,BBB建設,0,0.000000,False,0,0
2,A02,2025-01,材料費,150000,150000,AAA商事,0,0.000000,False,1,1
3,A02,2025-02,交通費,20000,18000,交通会社,2000,0.111111,True,1,1
4,A03,2025-01,材料費,180000,175000,CCC工業,5000,0.028571,True,2,2以上
5,A03,2025-02,外注費,250000,260000,BBB建設,-10000,-0.038462,True,2,2以上
6,A04,2025-01,交通費,15000,15000,交通会社,0,0.000000,False,1,1
7,A04,2025-02,材料費,160000,155000,AAA商事,5000,0.032258,True,1,1


In [17]:
# Power BI 用にCSV出力
df_merged.to_csv(
"keihi_powerbi_input.csv",
index=False,
encoding="utf-8-sig"
)

In [5]:
# 現場別の差異件数と差異金額合計
site_summary = df_check.groupby("現場ID").agg(
差異件数=("差異金額", "count"),
差異金額合計=("差異金額", "sum")
).reset_index()

site_summary

,現場ID,差異件数,差異金額合計
0,A02,1,2000
1,A03,2,-5000
2,A04,1,5000


In [18]:
# 取引先別の差異件数と差異金額合計
vendor_summary = df_check.groupby("取引先").agg(
差異件数=("差異金額", "count"),
差異金額合計=("差異金額", "sum")
).reset_index()

vendor_summary

,取引先,差異件数,差異金額合計
0,AAA商事,1,5000
1,BBB建設,1,-10000
2,CCC工業,1,5000
3,交通会社,1,2000


In [22]:
# CSVファイルとして保存
df.to_csv("keihi_all_data.csv", index=False)
df_check.to_csv("keihi_check_data.csv", index=False)
df_summary.to_csv("keihi_site_summary.csv", index=False)
vendor_summary.to_csv("keihi_vendor_summary.csv", index=False)

print("CSVファイルを書き出しました")

CSVファイルを書き出しました
